### **Estudo sobre decomposições matriciais**

---

Gabriel Oukawa <br>
Álgebra linear para ciência de dados <br>
2º Semestre de 2025

---


O presente relatório tem como objetivo realizar um estudo sobre diferentes decomposições matriciais. <br>
Na primeira parte, será analisado o comportamento dos seguintes métodos: QR, SVD e Cholesky. <br>
Na segunda parte, os métodos ST, TS e Conjugada serão implementados e também avaliados.

## **1. Decomposições QR, SVD e Cholesky** ##



In [ ]:
# Bibliotecas
import numpy as np
from scipy.linalg import pascal
from tabulate import tabulate

In [ ]:
# Precisão da máquina
eps = np.finfo(float).eps
print(eps)

2.220446049250313e-16


In [ ]:
# Algoritmo "e" (estabilidade)
def calc_e(X, A):
    # Norma 2 para todos os cálculos
    A = np.linalg.pinv(A)
    cond = np.linalg.cond(A, 2)
    num = np.linalg.norm(X - A, 2)
    den = eps * np.linalg.norm(A, 2) * cond
    return num / den

# Algoritmo "res" (erro residual)
def calc_res(X, A):
    # Norma 2 para todos os cálculos
    m, n = A.shape
    identity = np.eye(m)
    num = np.linalg.norm(X @ A - identity, 2)
    den = np.linalg.norm(A, 2) * np.linalg.norm(X, 2)
    return num / den

In [ ]:
# Decomposição Cholesky
def cholesky(A):
    M = A.T @ A # Calcular a triangular superior
    # Fatorização Cholesky
    try:
        R = np.linalg.cholesky(M, upper=True)
    except np.linalg.LinAlgError:
        print('Erro: A matriz não é SPD')
    Y = np.linalg.solve(R.T, A.T) # Resolver para Y
    X = np.linalg.solve(R, Y) # Resolver para X (inversa de Moore-Penrose)
    return X

In [ ]:
# Decomposição QR
def qr_mp(A):
    try:
        Q, R = np.linalg.qr(A, mode='reduced')
    except np.linalg.LinAlgError:
        print('Erro: A matriz não é de posto completo')
    # A inversa Moore-Penrose é a solução do sistema
    QT = Q.T
    X = np.linalg.solve(R, QT)
    return X

In [ ]:
# Decomposição SVD
def svd(A):
    # U e Vt são ortogonais, e S é uma matriz unidimensional
    U, S, Vt = np.linalg.svd(A)
    S_inv = np.diag(1 / S)
    return Vt.T @ S_inv @ U.T

In [ ]:
# Testes numéricos
n_runs = [4, 6, 8, 10]
results = {}

for n in n_runs:
    A = pascal(n) # Matriz tipo Pascal
    A_pinv, condA = np.linalg.pinv(A), np.linalg.cond(A)
    # Estabilidade
    errors = {
        "e_chol": calc_e(cholesky(A), A_pinv),
        "e_qr": calc_e(qr_mp(A), A_pinv),
        "e_svd": calc_e(svd(A), A_pinv)
    }
    # Erro residual
    residuals = {
        "res_chol": calc_res(cholesky(A), A),
        "res_qr": calc_res(qr_mp(A), A),
        "res_svd": calc_res(svd(A), A)
    }

    # Guardar resultados
    results[n] = {
        "cond": condA,
        "chol": {"e": errors["e_chol"], "res": residuals["res_chol"]},
        "qr":   {"e": errors["e_qr"], "res": residuals["res_qr"]},
        "svd":  {"e": errors["e_svd"], "res": residuals["res_svd"]}
    }

# Mostrar resultados
headers = ["n", "cond(A)",
           "e_chol", "res_chol",
           "e_qr", "res_qr",
           "e_svd", "res_svd"]
table = [[n, f"{r['cond']:.2e}",
          f"{r['chol']['e']:.2e}", f"{r['chol']['res']:.2e}",
          f"{r['qr']['e']:.2e}", f"{r['qr']['res']:.2e}",
          f"{r['svd']['e']:.2e}", f"{r['svd']['res']:.2e}"]
         for n,r in results.items()]

print(tabulate(table, headers=headers, tablefmt="pretty"))

+----+----------+----------+----------+----------+----------+----------+----------+
| n  | cond(A)  |  e_chol  | res_chol |   e_qr   |  res_qr  |  e_svd   | res_svd  |
+----+----------+----------+----------+----------+----------+----------+----------+
| 4  | 6.92e+02 | 6.50e+12 | 1.35e-14 | 6.50e+12 | 8.13e-17 | 6.50e+12 | 7.19e-17 |
| 6  | 1.11e+05 | 4.07e+10 | 1.29e-12 | 4.07e+10 | 2.51e-16 | 4.07e+10 | 2.66e-17 |
| 8  | 2.06e+07 | 2.18e+08 | 1.50e-10 | 2.18e+08 | 3.00e-16 | 2.18e+08 | 3.38e-17 |
| 10 | 4.16e+09 | 1.23e+06 | 2.88e-08 | 1.08e+06 | 6.95e-15 | 1.08e+06 | 8.88e-18 |
+----+----------+----------+----------+----------+----------+----------+----------+


A tabela acima mostra uma síntese dos resultados para uma matriz tipo Pascal. Todos os métodos apresentaram fatores de estabilidade semelhantes, com desempenho apenas ligeiramente inferior para o método de Cholesky em *n* = 10. Quanto ao erro residual, os métodos QR e SVD se destacaram, sendo o SVD mais consistente e superior para valores maiores de *n* (entre 8 e 10).

## **2. Decomposições ST, TS e Conjugada** ##

In [ ]:
# Bibliotecas
import numpy as np
from scipy.linalg import pascal
from tabulate import tabulate

In [ ]:
# Decomposições ST e TS, seguindo os algoritmos 3.1 e 3.2
def st(A):

    n = A.shape[0]
    T = np.zeros_like(A, dtype=float)
    L = np.zeros_like(A, dtype=float)

    # t11 e a11 devem ser > 0
    t11 = np.sign(A[0, 0]) if A[0, 0] != 0 else 1.0
    if t11 * A[0, 0] <= 0:
        t11 = -t11
    T[0, 0] = t11
    L[0, 0] = np.sqrt(t11 * A[0, 0])

    for k in range(n - 1):
        Lk = L[:k+1, :k+1]
        Tk = T[:k+1, :k+1]

        ak1 = A[:k+1, k+1]
        a1k = A[k+1, :k+1]
        akk = A[k+1, k+1]

        lk1 = np.linalg.solve(Lk, Tk @ ak1)

        lhat = np.linalg.solve(Lk, a1k)

        s = akk - lhat @ lk1

        tkk = 1.0 if s > 0 else -1.0
        gamma = tkk * s
        if gamma <= 0: # gamma deve ser > 0
            break

        L[k+1, :k+1] = lk1
        L[k+1, k+1] = np.sqrt(gamma)
        T[k+1, :k+1] = np.linalg.solve(Lk.T, (lk1 - tkk * lhat))
        T[k+1, k+1] = tkk

    return T, L

def ts(A):

    n = A.shape[0]
    T = np.zeros_like(A, dtype=float)
    L = np.zeros_like(A, dtype=float)

    # t11 deve ser > 0
    t11 = np.sign(A[0, 0]) if A[0, 0] != 0 else 1.0
    if t11 * A[0, 0] <= 0:
        t11 = -t11
    T[0, 0] = t11
    L[0, 0] = np.sqrt(t11 * A[0, 0])

    for k in range(n - 1):
        Lk = L[:k+1, :k+1]
        Tk = T[:k+1, :k+1]

        ak1 = A[:k+1, k+1]
        a1k = A[k+1, :k+1]
        akk = A[k+1, k+1]

        lk1 = np.linalg.solve(Lk, np.linalg.solve(Tk, ak1))

        lhat = np.linalg.solve(Lk, a1k)

        s = akk - lhat @ lk1

        tkk = 1.0 if s > 0 else -1.0
        gamma = s / tkk
        if gamma <= 0: # gamma deve ser > 0
            break

        L[k+1, :k+1] = lk1
        L[k+1, k+1] = np.sqrt(gamma)
        T[k+1, :k+1] = np.linalg.solve(Lk.T, (lk1 - tkk * lhat))
        T[k+1, k+1] = tkk

    return T, L

In [ ]:
# Decomposição conjugada
def cj(A):
    m, n = A.shape
    eigvals, eigvecs = np.linalg.eigh(A.T @ A)

    # Ordenar autovalores e autovetores
    idx = np.argsort(eigvals)[::-1]
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]

    k = np.sum(eigvals > 1e-16)
    P = eigvecs

    gammas = np.sqrt(np.clip(eigvals[:k], 0, None))

    # Construir Q
    Qk = np.column_stack([(A @ P[:, i]) / gammas[i] for i in range(k)])

    if k < m:
        Q_rest, _ = np.linalg.qr(np.random.randn(m, m - k))

        for i in range(Q_rest.shape[1]):
            for j in range(Qk.shape[1]):
                Q_rest[:, i] -= np.dot(Qk[:, j], Q_rest[:, i]) * Qk[:, j]
            Q_rest[:, i] /= np.linalg.norm(Q_rest[:, i])
        Q = np.column_stack((Qk, Q_rest))
    else:
        Q = Qk

    G = np.zeros((m, n))
    np.fill_diagonal(G, np.concatenate([gammas, np.zeros(min(m, n) - k)]))

    return Q, G, P

In [ ]:
n_runs = [2, 3, 4, 50, 100, 200, 500]
table = []
np.random.seed(42)

for n in n_runs:
    A_rand = np.random.rand(n, n)
    A = A_rand + n * np.eye(n)

    # ST (A = TLL^T)
    T_st, L_st = st(A)

    # TS (TA = LL^T)
    T_ts, L_ts = ts(A)

    # ST (A = TLL^T)
    X_st = T_st @ L_st @ L_st.T

    # TS (A = T^-1 * LL^T)
    X_ts = np.linalg.inv(T_ts) @ L_ts @ L_ts.T

    Q_cj, G_cj, P_cj = cj(A)

    X_cj = Q_cj @ G_cj @ np.linalg.inv(P_cj)

    # Erros de reconstrução - Norma 2 para todos os cálculos
    err_st = np.linalg.norm(A - X_st, 2) / np.linalg.norm(A, 2)
    err_ts = np.linalg.norm(A - X_ts, 2) / np.linalg.norm(A, 2)
    err_cj = np.linalg.norm(A - X_cj, 2) / np.linalg.norm(A, 2)

    row = [n, err_st, err_ts, err_cj]
    table.append(row)

# Mostrar resultados
print(tabulate(table, headers=["n", "erro_st", "erro_ts", "erro_cj"], tablefmt="pretty"))

+-----+---------------------+------------------------+------------------------+
|  n  |       erro_st       |        erro_ts         |        erro_cj         |
+-----+---------------------+------------------------+------------------------+
|  2  | 0.14117125901172195 | 1.3304900379139732e-16 | 2.036682758994826e-16  |
|  3  | 0.3051091798321605  |  0.010738898895361617  | 3.303461305977293e-16  |
|  4  | 0.2433008815550778  |  0.010491833115796096  | 2.5369353239302464e-16 |
| 50  | 0.11941825169355842 |  0.011139482390531162  | 2.122330612724927e-15  |
| 100 | 0.08620804927401127 |  0.00762005125729613   | 5.780908566304559e-15  |
| 200 | 0.06209257185812615 |  0.005724850467223399  | 1.356784111112832e-14  |
| 500 | 0.04013613736332133 | 0.0035381045187833104  |  4.90575798700282e-14  |
+-----+---------------------+------------------------+------------------------+


A tabela acima mostra uma síntese dos resultados para uma matriz quadrada (*n* x *n*) onde os elementos da diagonal principal são maiores que *n* e os elementos fora da diagonal estão entre 0 e 1. <br>
Aqui, testamos somente o erro de reconstrução, usando a norma 2. Para todos os tamanhos, o método conjugado foi o melhor, apresentando erros muito pequenos mesmo para matrizes grandes (por exemplo, *n* > 100). Os métodos ST e TS tiveram um desempenho similar, com a exceção de matrizes pequenas, onde o TS foi melhor.